In [1]:
%matplotlib qt5
import hyperspy.api as hs
import pyxem as pxm
import numpy as np
import orix
import matplotlib.pyplot as plt

from pathlib import Path
from diffpy.structure import Atom, Lattice, Structure
from diffsims.generators.simulation_generator import SimulationGenerator
from orix.vector import Miller, Vector3d

In [22]:
#file_path = Path("D:/datasets/mini_centered.hspy")
file_path = Path("D:/datasets/cropped_Ag_centered.hspy")
file_path3 = Path("D:/datasets/SPED_Ag_full_centered.hspy")

dp_mini_centered = hs.load(file_path, lazy=True)
#dp_cropped_centered = hs.load(file_path2, lazy=True)
dp_centered = hs.load(file_path3, lazy=True)

In [23]:
dp_centered.plot()

  0%|          | 0/501 [00:00<?, ?it/s]

In [3]:
a = 4.08 # lattice parameter for silver in Å
atoms = [Atom('Ag', [0, 0, 0]), Atom('Ag', [0.5, 0.5, 0]), Atom('Ag', [0.5, 0, 0.5]), Atom('Ag', [0, 0.5, 0.5])]
lattice = Lattice(a, a, a, 90, 90, 90)
phase = orix.crystal_map.Phase(name='Ag', space_group=225, structure=Structure(atoms, lattice))

angular_resolution = 0.5
grid = orix.sampling.get_sample_reduced_fundamental(angular_resolution, point_group=phase.point_group)
orientations = orix.quaternion.Orientation(grid, symmetry=phase.point_group)

simgen = SimulationGenerator(precession_angle=1., minimum_intensity=1e-5)
simulations = simgen.calculate_diffraction2d(phase=phase, rotation=grid, reciprocal_radius=1.5, with_direct_beam=False, max_excitation_error=0.01)

In [4]:
ny, nx = dp_mini_centered.axes_manager.signal_shape
center = (ny/2 - 0.5, nx/2 - 0.5)
dp_mini_centered.change_dtype('float32')
dp_mini_centered.calibration(center=center)

azi = dp_mini_centered.get_azimuthal_integral2d(npt=256)

In [5]:
results = azi.get_orientation(simulations, n_best=grid.size, frac_keep=1.0)

dp_mini_centered.plot(cmap='viridis_r', norm='log')
dp_mini_centered.add_marker(results.to_markers(annotate=True))

WARNING | Hyperspy | The function you applied does not take into account the difference of scales in-between axes. (hyperspy.signal:5487)
WARNING | Hyperspy | The function you applied does not take into account the difference of units in-between axes. (hyperspy.signal:5492)


  0%|          | 0/101 [00:00<?, ?it/s]

WARNING | Hyperspy | Using `navigaion_chunk=None` is deprecated, `navigaion_chunk='auto'` is used instead. (hyperspy.signal:5418)
WARNING | Hyperspy | Using `navigaion_chunk=None` is deprecated, `navigaion_chunk='auto'` is used instead. (hyperspy.signal:5418)


In [6]:
ix, iy = dp_mini_centered.axes_manager.indices# 2, 2 # 0
print(f'Plotting results for navigation position ({ix:.0f}, {iy:.0f})')

correlations_i = results.inav[ix, iy].data[:, 1]
template_indices_i = (results.inav[ix, iy].data[:, 0]).astype('int16')
orientations_i = orientations[template_indices_i]
euler_angles_i = orientations_i.to_euler()

fig = plt.figure()
ax = fig.add_subplot(111, projection="ipf", symmetry=phase.point_group)
ax.scatter(orientations_i, c=correlations_i, cmap='inferno')
ax.set_title(f'Correlation for label ({ix}, {iy})')

# Print the hkl indices of the best matching simulation:
sim_i = simulations.get_simulation(template_indices_i[0])
sim_i

Plotting results for navigation position (0, 0)


(Rotation (1,)
 [[ 0.6533 -0.2706  0.2706 -0.6533]],
 0,
 DiffractingVector (42,), Ag (m-3m)
 [[ 4.  2. -4.]
  [ 4.  0. -4.]
  [ 4. -2. -4.]
  [ 3.  3. -3.]
  [ 3.  1. -3.]
  [ 3. -1. -3.]
  [ 3. -3. -3.]
  [ 2.  4. -2.]
  [ 2.  2. -2.]
  [ 2. -0. -2.]
  [ 2. -2. -2.]
  [ 2. -4. -2.]
  [ 1.  5. -1.]
  [ 1.  3. -1.]
  [ 1.  1. -1.]
  [ 1. -1. -1.]
  [ 1. -3. -1.]
  [ 1. -5. -1.]
  [-0.  6. -0.]
  [-0.  4. -0.]
  [ 0.  2. -0.]
  [-0. -2.  0.]
  [ 0. -4.  0.]
  [ 0. -6.  0.]
  [-1.  5.  1.]
  [-1.  3.  1.]
  [-1.  1.  1.]
  [-1. -1.  1.]
  [-1. -3.  1.]
  [-1. -5.  1.]
  [-2.  4.  2.]
  [-2.  2.  2.]
  [-2.  0.  2.]
  [-2. -2.  2.]
  [-2. -4.  2.]
  [-3.  3.  3.]
  [-3.  1.  3.]
  [-3. -1.  3.]
  [-3. -3.  3.]
  [-4.  2.  4.]
  [-4.  0.  4.]
  [-4. -2.  4.]])

In [7]:
loris = results.to_single_phase_orientations()

In [13]:
loris_best = loris[[1,50,-1], [0,50,-1], 0]
loris_best

Orientation (3,) m-3m
[[ 0.3235  0.3584  0.134   0.8654]
 [ 0.5242 -0.3121  0.2145 -0.7628]
 [ 0.2395 -0.3657  0.098  -0.894 ]]

In [14]:
loris_best.scatter('ipf')
loris_best.scatter()
loris_euler = loris_best.to_euler(degrees=True)
loris_euler

array([[131. ,  45. ,  90. ],
       [ 21. ,  44.5,  90. ],
       [ 60. ,  44.5,  90. ]])

In [16]:
loris_ang = loris_best.angle_with_outer(loris_best, degrees=True)
loris_ang

array([[ 0.        , 59.71660793, 59.71259236],
       [59.71660793,  0.        , 39.        ],
       [59.71259236, 39.        ,  0.        ]])

In [17]:
plt.figure()
plt.hist(loris_ang.flatten(), bins=60);

In [18]:
dp_template = dp_mini_centered.template_match_disk(disk_r=3, subtract_min=False)
dp_template_peaks = dp_template.get_diffraction_vectors(min_distance=3, threshold_abs=0.6)
markers = dp_template_peaks.to_markers(sizes=3, color='cyan')

dp_mini_centered.plot(cmap='magma', norm='log')
dp_mini_centered.add_marker(markers)

WARNING | Hyperspy | The function you applied does not take into account the difference of scales in-between axes. (hyperspy.signal:5487)
WARNING | Hyperspy | The function you applied does not take into account the difference of units in-between axes. (hyperspy.signal:5492)


In [19]:
vectors_mask = dp_template_peaks.to_mask(disk_r=3)
dp_mini_centered_masked = dp_mini_centered * vectors_mask

dp_masked_pol = dp_mini_centered_masked.get_azimuthal_integral2d(npt=256)
dp_masked_results = dp_masked_pol.get_orientation(simulations, n_best=1, frac_keep=0.5)
dp_masked_results.compute()
xmap = dp_masked_results.to_crystal_map()
oris = xmap.orientations
corrs = dp_masked_results.data[:,:,0,1].flatten()

WARNING | Hyperspy | The function you applied does not take into account the difference of scales in-between axes. (hyperspy.signal:5487)
WARNING | Hyperspy | The function you applied does not take into account the difference of units in-between axes. (hyperspy.signal:5492)


  0%|          | 0/151 [00:00<?, ?it/s]

  0%|          | 0/145 [00:00<?, ?it/s]

In [20]:
from orix.plot import IPFColorKeyTSL
key_x = IPFColorKeyTSL(phase.point_group, Vector3d.xvector())
key_y = IPFColorKeyTSL(phase.point_group, Vector3d.yvector())
key_z = IPFColorKeyTSL(phase.point_group, Vector3d.zvector())

oris_z = key_z.orientation2color(oris)[:,0,:]
xmap.plot(oris_z, overlay=corrs, remove_padding=True)
oris_x = key_x.orientation2color(oris)[:,0,:]
xmap.plot(oris_x, overlay=corrs, remove_padding=True)
oris_y = key_y.orientation2color(oris)[:,0,:]
xmap.plot(oris_y, overlay=corrs, remove_padding=True)